In [1]:
import keras
import numpy as np
import tensorflow as tf

from absl import flags
from pathlib import Path
from operator import itemgetter
from keras.optimizers import Adam
from sklearn.metrics import classification_report, recall_score, precision_score, f1_score

Using TensorFlow backend.


In [2]:
import importlib

import model as nn
importlib.reload(nn)

elsa_architecture = nn.elsa_architecture

In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "2"

lang = "en"
batch_size = 250
lr = 1e-3
epochs = 100
patience = 3
data_dir = "/data/elsa2"
checkpoint_dir = "./ckpt"
optimizer = "adam"

In [4]:
lstm_hidden = 512
lstm_drop = 0.5
final_drop = 0.5
embed_drop = 0.0
highway = False
compute_class_weight = False
multilabel = True

In [5]:
data_dir = Path(data_dir)
wv_path = (data_dir / "{:s}_wv.npy".format(lang)).__str__()
X_path = (data_dir / "{:s}_X.npy".format(lang)).__str__()
y_path = (data_dir / "{:s}_y.npy".format(lang)).__str__()
emoji_path = (data_dir / "{:s}_emoji.txt".format(lang)).__str__()

wv = np.load(wv_path, allow_pickle=True)
input_vec = np.load(X_path, allow_pickle=True)
input_label = np.load(y_path, allow_pickle=True)

nb_tokens = len(wv)
embed_dim = wv.shape[1]
input_len = len(input_label)
nb_classes = input_label.shape[1]
maxlen = input_vec.shape[1]

train_end = int(input_len*0.7)
val_end = int(input_len*0.9)

(X_train, y_train) = (input_vec[:train_end], input_label[:train_end])
(X_val, y_val) = (input_vec[train_end:val_end], input_label[train_end:val_end])
(X_test, y_test) = (input_vec[val_end:], input_label[val_end:])

if multilabel:
    def to_multilabel(y):
        outputs = []
        for i in range(nb_classes):
            outputs.append(y[:, i])
        return outputs

    y_train = to_multilabel(y_train)
    y_val = to_multilabel(y_val)
    y_test = to_multilabel(y_test)

model = elsa_architecture(nb_classes=nb_classes,
                          nb_tokens=nb_tokens,
                          maxlen=maxlen,
                          final_dropout_rate=final_drop,
                          embed_dropout_rate=embed_drop,
                          load_embedding=True,
                          pre_embedding=wv,
                          high=highway,
                          embed_dim=embed_dim,
                          multilabel=multilabel)
model.summary()

computed_class_weight = None

if multilabel:
    loss = "binary_crossentropy"
else:
    loss = "categorical_crossentropy"
    if compute_class_weight:
        y_train_sps = []
        for row in y_train:
            y_train_sps.extend(np.where(row)[0].tolist())
        computed_class_weight = class_weight.compute_class_weight(
            'balanced', list(range(nb_classes)), y_train_sps)
        print("computed class weight = {:s}".format(str(computed_class_weight)))

if optimizer == 'adam':
    adam = Adam(clipnorm=1, lr=lr)
    model.compile(loss=loss, optimizer=adam, metrics=['accuracy'])
elif optimizer == 'rmsprop':
    model.compile(loss=loss, optimizer='rmsprop', metrics=['accuracy'])

checkpoint_dir = Path(checkpoint_dir)
if not checkpoint_dir.exists():
    checkpoint_dir.mkdir()
checkpoint_weight_path = (checkpoint_dir / "elsa_{:s}.hdf5".format(lang)).__str__()

callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_loss', min_delta=0, patience=patience, verbose=0, mode='auto'),
    keras.callbacks.ModelCheckpoint(checkpoint_weight_path, monitor='val_loss',
                                    verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
]
model.fit(X_train,
          y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(X_val, y_val),
          class_weight=computed_class_weight,
          callbacks=callbacks,
          verbose=1)

freq = {line.split()[0]: int(line.split()[1]) for line in open(emoji_path).readlines()}
freq_topn = sorted(freq.items(), key=itemgetter(1), reverse=True)[:nb_classes]

if multilabel:
    y_pred = model.predict([X_test], batch_size=batch_size)
    y_pred = [np.squeeze(p) for p in y_pred]

    y_test_1d = np.array(y_test).flatten()
    y_pred_1d = np.array(y_pred).flatten()
    print(f1_score(y_test_1d, y_pred_1d > 0.5))
    print(classification_report(y_test_1d, y_pred_1d > 0.5))

    gold, pred = [], []
    for i in range(len(X_test)):
        each_gold, each_pred = [], []
        for c in range(nb_classes):
            if y_test[c][i] == 1.0:
                each_gold.append(c+1)
            else:
                each_gold.append(0)
            if y_pred[c][i] > 0.5:
                each_pred.append(c+1)
            else:
                each_pred.append(0)
        gold.extend(each_gold)
        pred.extend(each_pred)

    target_name = [""] + [e[0] for e in freq_topn]
    print(classification_report(gold, pred, target_names=target_name))
else:
    _, acc = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)
    print(acc)

    y_pred = model.predict(X_test)
    print(classification_report(y_test.argmax(axis=1), y_pred.argmax(
        axis=1), target_names=[e[0] for e in freq_topn]))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 20, 200)      64132000    input_1[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 20, 200)      0           embedding[0][0]                  
__________________________________________________________________________________________________
bi_lstm_0 (Bidirectio

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 2677236 samples, validate on 764924 samples
Epoch 1/100
2677236/2677236 [==============================] - 3522s 1ms/step - loss: 5.3786 - sigmoid_0_loss: 0.4244 - sigmoid_1_loss: 0.3062 - sigmoid_2_loss: 0.2592 - sigmoid_3_loss: 0.1894 - sigmoid_4_loss: 0.1954 - sigmoid_5_loss: 0.1327 - sigmoid_6_loss: 0.1142 - sigmoid_7_loss: 0.1017 - sigmoid_8_loss: 0.0824 - sigmoid_9_loss: 0.1276 - sigmoid_10_loss: 0.0879 - sigmoid_11_loss: 0.1182 - sigmoid_12_loss: 0.0718 - sigmoid_13_loss: 0.0621 - sigmoid_14_loss: 0.1098 - sigmoid_15_loss: 0.1057 - sigmoid_16_loss: 0.0665 - sigmoid_17_loss: 0.1028 - sigmoid_18_loss: 0.0711 - sigmoid_19_loss: 0.0884 - sigmoid_20_loss: 0.0893 - sigmoid_21_loss: 0.0755 - sigmoid_22_loss: 0.0874 - sigmoid_23_loss: 0.0766 - sigmoid_24_loss: 0.0805 - sigmoid_25_loss: 0.0763 - sigmoid_26_loss: 0.0821 - sigmoid_27_loss: 0.0777 - sigmoid_

    500/2677236 [..............................] - ETA: 59:24 - loss: 5.0260 - sigmoid_0_loss: 0.4097 - sigmoid_1_loss: 0.3225 - sigmoid_2_loss: 0.2689 - sigmoid_3_loss: 0.1862 - sigmoid_4_loss: 0.2006 - sigmoid_5_loss: 0.1232 - sigmoid_6_loss: 0.0769 - sigmoid_7_loss: 0.1028 - sigmoid_8_loss: 0.0515 - sigmoid_9_loss: 0.1001 - sigmoid_10_loss: 0.0491 - sigmoid_11_loss: 0.1139 - sigmoid_12_loss: 0.0450 - sigmoid_13_loss: 0.0343 - sigmoid_14_loss: 0.0719 - sigmoid_15_loss: 0.0838 - sigmoid_16_loss: 0.0495 - sigmoid_17_loss: 0.0979 - sigmoid_18_loss: 0.0626 - sigmoid_19_loss: 0.0502 - sigmoid_20_loss: 0.0884 - sigmoid_21_loss: 0.0672 - sigmoid_22_loss: 0.0648 - sigmoid_23_loss: 0.0932 - sigmoid_24_loss: 0.0732 - sigmoid_25_loss: 0.0918 - sigmoid_26_loss: 0.0551 - sigmoid_27_loss: 0.0794 - sigmoid_28_loss: 0.0675 - sigmoid_29_loss: 0.0514 - sigmoid_30_loss: 0.0928 - sigmoid_31_loss: 0.0295 - sigmoid_32_loss: 0.0976 - sigmoid_33_loss: 0.0263 - sigmoid_34_loss: 0.0840 - sigmoid_35_loss: 0.10

2677236/2677236 [==============================] - 3190s 1ms/step - loss: 5.1060 - sigmoid_0_loss: 0.4096 - sigmoid_1_loss: 0.2959 - sigmoid_2_loss: 0.2491 - sigmoid_3_loss: 0.1837 - sigmoid_4_loss: 0.1854 - sigmoid_5_loss: 0.1267 - sigmoid_6_loss: 0.1058 - sigmoid_7_loss: 0.0933 - sigmoid_8_loss: 0.0728 - sigmoid_9_loss: 0.1226 - sigmoid_10_loss: 0.0794 - sigmoid_11_loss: 0.1139 - sigmoid_12_loss: 0.0639 - sigmoid_13_loss: 0.0568 - sigmoid_14_loss: 0.1054 - sigmoid_15_loss: 0.1013 - sigmoid_16_loss: 0.0569 - sigmoid_17_loss: 0.0965 - sigmoid_18_loss: 0.0607 - sigmoid_19_loss: 0.0818 - sigmoid_20_loss: 0.0856 - sigmoid_21_loss: 0.0704 - sigmoid_22_loss: 0.0829 - sigmoid_23_loss: 0.0736 - sigmoid_24_loss: 0.0767 - sigmoid_25_loss: 0.0720 - sigmoid_26_loss: 0.0757 - sigmoid_27_loss: 0.0750 - sigmoid_28_loss: 0.0713 - sigmoid_29_loss: 0.0560 - sigmoid_30_loss: 0.0717 - sigmoid_31_loss: 0.0637 - sigmoid_32_loss: 0.0634 - sigmoid_33_loss: 0.0304 - sigmoid_34_loss: 0.0674 - sigmoid_35_loss: 

    500/2677236 [..............................] - ETA: 1:02:40 - loss: 5.0077 - sigmoid_0_loss: 0.3845 - sigmoid_1_loss: 0.2547 - sigmoid_2_loss: 0.2649 - sigmoid_3_loss: 0.1754 - sigmoid_4_loss: 0.1745 - sigmoid_5_loss: 0.1391 - sigmoid_6_loss: 0.1135 - sigmoid_7_loss: 0.0710 - sigmoid_8_loss: 0.0665 - sigmoid_9_loss: 0.1376 - sigmoid_10_loss: 0.0560 - sigmoid_11_loss: 0.1302 - sigmoid_12_loss: 0.0566 - sigmoid_13_loss: 0.0541 - sigmoid_14_loss: 0.1057 - sigmoid_15_loss: 0.1047 - sigmoid_16_loss: 0.0592 - sigmoid_17_loss: 0.0963 - sigmoid_18_loss: 0.0627 - sigmoid_19_loss: 0.0726 - sigmoid_20_loss: 0.0778 - sigmoid_21_loss: 0.0720 - sigmoid_22_loss: 0.0650 - sigmoid_23_loss: 0.0635 - sigmoid_24_loss: 0.0964 - sigmoid_25_loss: 0.0971 - sigmoid_26_loss: 0.1330 - sigmoid_27_loss: 0.0817 - sigmoid_28_loss: 0.0695 - sigmoid_29_loss: 0.0321 - sigmoid_30_loss: 0.0819 - sigmoid_31_loss: 0.0547 - sigmoid_32_loss: 0.0783 - sigmoid_33_loss: 0.0324 - sigmoid_34_loss: 0.0640 - sigmoid_35_loss: 0.

...

2677236/2677236 [==============================] - 3605s 1ms/step - loss: 4.5072 - sigmoid_0_loss: 0.3668 - sigmoid_1_loss: 0.2688 - sigmoid_2_loss: 0.2222 - sigmoid_3_loss: 0.1656 - sigmoid_4_loss: 0.1616 - sigmoid_5_loss: 0.1149 - sigmoid_6_loss: 0.0925 - sigmoid_7_loss: 0.0784 - sigmoid_8_loss: 0.0498 - sigmoid_9_loss: 0.1074 - sigmoid_10_loss: 0.0673 - sigmoid_11_loss: 0.1026 - sigmoid_12_loss: 0.0444 - sigmoid_13_loss: 0.0472 - sigmoid_14_loss: 0.0929 - sigmoid_15_loss: 0.0894 - sigmoid_16_loss: 0.0358 - sigmoid_17_loss: 0.0830 - sigmoid_18_loss: 0.0385 - sigmoid_19_loss: 0.0686 - sigmoid_20_loss: 0.0759 - sigmoid_21_loss: 0.0603 - sigmoid_22_loss: 0.0707 - sigmoid_23_loss: 0.0664 - sigmoid_24_loss: 0.0690 - sigmoid_25_loss: 0.0633 - sigmoid_26_loss: 0.0630 - sigmoid_27_loss: 0.0653 - sigmoid_28_loss: 0.0609 - sigmoid_29_loss: 0.0463 - sigmoid_30_loss: 0.0611 - sigmoid_31_loss: 0.0554 - sigmoid_32_loss: 0.0537 - sigmoid_33_loss: 0.0253 - sigmoid_34_loss: 0.0593 - sigmoid_35_loss: 

    500/2677236 [..............................] - ETA: 1:03:29 - loss: 4.3578 - sigmoid_0_loss: 0.3231 - sigmoid_1_loss: 0.2446 - sigmoid_2_loss: 0.2284 - sigmoid_3_loss: 0.1483 - sigmoid_4_loss: 0.1673 - sigmoid_5_loss: 0.1509 - sigmoid_6_loss: 0.0901 - sigmoid_7_loss: 0.1039 - sigmoid_8_loss: 0.0403 - sigmoid_9_loss: 0.0863 - sigmoid_10_loss: 0.0640 - sigmoid_11_loss: 0.1379 - sigmoid_12_loss: 0.0402 - sigmoid_13_loss: 0.0679 - sigmoid_14_loss: 0.1130 - sigmoid_15_loss: 0.0955 - sigmoid_16_loss: 0.0188 - sigmoid_17_loss: 0.0846 - sigmoid_18_loss: 0.0248 - sigmoid_19_loss: 0.0597 - sigmoid_20_loss: 0.0703 - sigmoid_21_loss: 0.0324 - sigmoid_22_loss: 0.0686 - sigmoid_23_loss: 0.0622 - sigmoid_24_loss: 0.0653 - sigmoid_25_loss: 0.0467 - sigmoid_26_loss: 0.0522 - sigmoid_27_loss: 0.0611 - sigmoid_28_loss: 0.0755 - sigmoid_29_loss: 0.0333 - sigmoid_30_loss: 0.0171 - sigmoid_31_loss: 0.0261 - sigmoid_32_loss: 0.0431 - sigmoid_33_loss: 0.0118 - sigmoid_34_loss: 0.0364 - sigmoid_35_loss: 0.

2677236/2677236 [==============================] - 3960s 1ms/step - loss: 4.5006 - sigmoid_0_loss: 0.3660 - sigmoid_1_loss: 0.2684 - sigmoid_2_loss: 0.2218 - sigmoid_3_loss: 0.1655 - sigmoid_4_loss: 0.1615 - sigmoid_5_loss: 0.1150 - sigmoid_6_loss: 0.0923 - sigmoid_7_loss: 0.0783 - sigmoid_8_loss: 0.0495 - sigmoid_9_loss: 0.1071 - sigmoid_10_loss: 0.0670 - sigmoid_11_loss: 0.1025 - sigmoid_12_loss: 0.0441 - sigmoid_13_loss: 0.0472 - sigmoid_14_loss: 0.0926 - sigmoid_15_loss: 0.0893 - sigmoid_16_loss: 0.0359 - sigmoid_17_loss: 0.0827 - sigmoid_18_loss: 0.0385 - sigmoid_19_loss: 0.0686 - sigmoid_20_loss: 0.0759 - sigmoid_21_loss: 0.0601 - sigmoid_22_loss: 0.0705 - sigmoid_23_loss: 0.0663 - sigmoid_24_loss: 0.0689 - sigmoid_25_loss: 0.0631 - sigmoid_26_loss: 0.0628 - sigmoid_27_loss: 0.0653 - sigmoid_28_loss: 0.0607 - sigmoid_29_loss: 0.0462 - sigmoid_30_loss: 0.0611 - sigmoid_31_loss: 0.0555 - sigmoid_32_loss: 0.0536 - sigmoid_33_loss: 0.0254 - sigmoid_34_loss: 0.0593 - sigmoid_35_loss: 

    500/2677236 [..............................] - ETA: 1:10:27 - loss: 4.3203 - sigmoid_0_loss: 0.3765 - sigmoid_1_loss: 0.2324 - sigmoid_2_loss: 0.2042 - sigmoid_3_loss: 0.1868 - sigmoid_4_loss: 0.1714 - sigmoid_5_loss: 0.1089 - sigmoid_6_loss: 0.0727 - sigmoid_7_loss: 0.0595 - sigmoid_8_loss: 0.0561 - sigmoid_9_loss: 0.0840 - sigmoid_10_loss: 0.0340 - sigmoid_11_loss: 0.1221 - sigmoid_12_loss: 0.0677 - sigmoid_13_loss: 0.0236 - sigmoid_14_loss: 0.0968 - sigmoid_15_loss: 0.0987 - sigmoid_16_loss: 0.0457 - sigmoid_17_loss: 0.0609 - sigmoid_18_loss: 0.0497 - sigmoid_19_loss: 0.0762 - sigmoid_20_loss: 0.0672 - sigmoid_21_loss: 0.0452 - sigmoid_22_loss: 0.0606 - sigmoid_23_loss: 0.0828 - sigmoid_24_loss: 0.0525 - sigmoid_25_loss: 0.0543 - sigmoid_26_loss: 0.0380 - sigmoid_27_loss: 0.0577 - sigmoid_28_loss: 0.0237 - sigmoid_29_loss: 0.0320 - sigmoid_30_loss: 0.0460 - sigmoid_31_loss: 0.0653 - sigmoid_32_loss: 0.0611 - sigmoid_33_loss: 0.0287 - sigmoid_34_loss: 0.0383 - sigmoid_35_loss: 0.

2677236/2677236 [==============================] - 3966s 1ms/step - loss: 4.4937 - sigmoid_0_loss: 0.3654 - sigmoid_1_loss: 0.2681 - sigmoid_2_loss: 0.2216 - sigmoid_3_loss: 0.1653 - sigmoid_4_loss: 0.1613 - sigmoid_5_loss: 0.1147 - sigmoid_6_loss: 0.0923 - sigmoid_7_loss: 0.0781 - sigmoid_8_loss: 0.0490 - sigmoid_9_loss: 0.1070 - sigmoid_10_loss: 0.0670 - sigmoid_11_loss: 0.1024 - sigmoid_12_loss: 0.0438 - sigmoid_13_loss: 0.0471 - sigmoid_14_loss: 0.0926 - sigmoid_15_loss: 0.0892 - sigmoid_16_loss: 0.0355 - sigmoid_17_loss: 0.0827 - sigmoid_18_loss: 0.0381 - sigmoid_19_loss: 0.0684 - sigmoid_20_loss: 0.0756 - sigmoid_21_loss: 0.0600 - sigmoid_22_loss: 0.0704 - sigmoid_23_loss: 0.0663 - sigmoid_24_loss: 0.0690 - sigmoid_25_loss: 0.0629 - sigmoid_26_loss: 0.0625 - sigmoid_27_loss: 0.0652 - sigmoid_28_loss: 0.0606 - sigmoid_29_loss: 0.0460 - sigmoid_30_loss: 0.0610 - sigmoid_31_loss: 0.0554 - sigmoid_32_loss: 0.0535 - sigmoid_33_loss: 0.0254 - sigmoid_34_loss: 0.0591 - sigmoid_35_loss: 